<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install numpy

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Write your code here
# Detailed summary of datatypes and non-null counts
print(df.info())

# Create a summary dataframe
summary = pd.DataFrame({
    'DataType': df.dtypes,
    'TotalCount': df.shape[0],
    'Non-Null Count': df.count(),
    'Missing Values': df.isnull().sum(),
    'Percentage Missing (%)': (df.isnull().sum() / len(df)) * 100
})

print(summary)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB
None
                    DataType  TotalCount  Non-Null Count  Missing Values  \
ResponseId             int64       65437           65437               0   
MainBranch            object       65437           65437               0   
Age                   object       65437           65437               0   
Employment            object       65437           65437               0   
RemoteWork            object       65437           54806           10631   
...                      ...         ...             ...             ...   
JobSatPoints_11      float64       65437           29445           35992   
SurveyLength          object       65437           56182            9255   
SurveyEase            object       65437           56238            9199   
ConvertedCompYearly  float64       65437          

<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Write your code here
numeric_stats = df.describe()

print("Summary Statistics for Numerical Columns:")
print(numeric_stats)

Summary Statistics for Numerical Columns:
         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Write your code here
# Get unique countries sorted by frequency
country_counts = df['Country'].value_counts()

# Print the top 20 and the bottom 20 (where inconsistencies usually hide)
print("Top 20 Countries:")
print(country_counts.head(20))

print("\nBottom 20 Countries (Check for typos/rare entries):")
print(country_counts.tail(20))

# Find countries that are too short (likely codes or typos)
short_entries = df[df['Country'].str.len() < 4]['Country'].unique()

# Find countries containing numbers
numeric_entries = df[df['Country'].str.contains(r'\d', na=False)]['Country'].unique()

print(f"Suspiciously short entries: {short_entries}")
print(f"Entries containing numbers: {numeric_entries}")

Top 20 Countries:
Country
United States of America                                11095
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
Ukraine                                                  2672
France                                                   2110
Canada                                                   2104
Poland                                                   1534
Netherlands                                              1449
Brazil                                                   1375
Italy                                                    1341
Australia                                                1260
Spain                                                    1123
Sweden                                                   1016
Russian Federation                                        925
Switzerland                                 

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [6]:
## Write your code here
# Standardizing the names
mapping = {
    'USA': 'United States',
    'US': 'United States',
    'United States of America': 'United States',
    'UK': 'United Kingdom',
    'germany': 'Germany'
}

df['Country'] = df['Country'].replace(mapping)

# Remove irrelevant entries (e.g., placeholder values)
irrelevant = ['Other', 'Unknown', '0']
df = df[~df['Country'].isin(irrelevant)]


### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [7]:
## Write your code here
# Perform One-Hot Encoding
employment_encoded = pd.get_dummies(df['Employment'], prefix='Employment')

# Join the new columns back to the original DataFrame
df = pd.concat([df, employment_encoded], axis=1)

# Drop the original 'Employment' column as it is now redundant
df.drop('Employment', axis=1, inplace=True)

# Save the transformed data to a new CSV
df.to_csv("encoded_survey_data.csv", index=False)

# Display the first few rows of the new columns
print("New One-Hot Encoded columns:")
print(employment_encoded.head())

New One-Hot Encoded columns:
   Employment_Employed, full-time  \
0                            True   
1                            True   
2                            True   
3                           False   
4                           False   

   Employment_Employed, full-time;Employed, part-time  \
0                                              False    
1                                              False    
2                                              False    
3                                              False    
4                                              False    

   Employment_Employed, full-time;Independent contractor, freelancer, or self-employed  \
0                                              False                                     
1                                              False                                     
2                                              False                                     
3                                            

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [8]:
## Write your code here
# Identify missing values per column
missing_data = df.isnull().sum()

# Sort the columns by the number of missing values (highest first)
top_missing = missing_data.sort_values(ascending=False)

# Display the top 10 columns with the most missing values
print("Columns with the highest number of missing values:")
print(top_missing.head(10))

Columns with the highest number of missing values:
AINextMuch less integrated       64289
AINextLess integrated            63082
AINextNo change                  52939
AINextMuch more integrated       51999
EmbeddedAdmired                  48704
EmbeddedWantToWorkWith           47837
EmbeddedHaveWorkedWith           43223
ConvertedCompYearly              42002
AIToolNot interested in Using    41023
AINextMore integrated            41009
dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [9]:
## Write your code here
# Identify numerical columns
# This selects columns with float or integer datatypes
numeric_cols = df.select_dtypes(include=['number']).columns

print(f"Numerical columns found: {list(numeric_cols)}")

# Impute missing values with the mean for each numeric column
for col in numeric_cols:
    mean_value = df[col].mean()
    df[col] = df[col].fillna(mean_value)

# Verify that there are no remaining missing values in these columns
missing_after = df[numeric_cols].isnull().sum()
print("\nMissing values after imputation:")
print(missing_after)

Numerical columns found: ['ResponseId', 'CompTotal', 'WorkExp', 'JobSatPoints_1', 'JobSatPoints_4', 'JobSatPoints_5', 'JobSatPoints_6', 'JobSatPoints_7', 'JobSatPoints_8', 'JobSatPoints_9', 'JobSatPoints_10', 'JobSatPoints_11', 'ConvertedCompYearly', 'JobSat']

Missing values after imputation:
ResponseId             0
CompTotal              0
WorkExp                0
JobSatPoints_1         0
JobSatPoints_4         0
JobSatPoints_5         0
JobSatPoints_6         0
JobSatPoints_7         0
JobSatPoints_8         0
JobSatPoints_9         0
JobSatPoints_10        0
JobSatPoints_11        0
ConvertedCompYearly    0
JobSat                 0
dtype: int64


<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [10]:
## Write your code here
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

print(f"Categorical columns identified: {list(categorical_cols)}")

# Impute with the most frequent value (Mode)
for col in categorical_cols:
    # .mode() returns a Series (in case of ties), so we take the first item [0]
    majority_value = df[col].mode()[0]
    df[col] = df[col].fillna(majority_value)

# Verify the results
remaining_missing = df[categorical_cols].isnull().sum().sum()
print(f"Total missing values in categorical columns after imputation: {remaining_missing}")

Categorical columns identified: ['MainBranch', 'Age', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith', 'OfficeS

### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [11]:
## Write your code here
# Handle Missing Values
median_val = df['ConvertedCompYearly'].median()
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].fillna(median_val)

# Apply Min-Max Scaling
min_val = df['ConvertedCompYearly'].min()
max_val = df['ConvertedCompYearly'].max()

df['ConvertedCompYearly_Normalized'] = (df['ConvertedCompYearly'] - min_val) / (max_val - min_val)

# Verify the transformation
print("First 5 rows of original vs normalized compensation:")
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_Normalized']].head())

First 5 rows of original vs normalized compensation:
   ConvertedCompYearly  ConvertedCompYearly_Normalized
0         86155.287263                          0.0053
1         86155.287263                          0.0053
2         86155.287263                          0.0053
3         86155.287263                          0.0053
4         86155.287263                          0.0053


/tmp/ipykernel_1474/689987053.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ConvertedCompYearly_Normalized'] = (df['ConvertedCompYearly'] - min_val) / (max_val - min_val)


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [12]:
## Write your code here
import numpy as np
# Log-transform requires non-null values.
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].fillna(df['ConvertedCompYearly'].median())

# Apply Log Transformation (using log1p which is log(1+x))
df['ConvertedComp_Log'] = np.log1p(df['ConvertedCompYearly'])

# Compare results
print(df[['ConvertedCompYearly', 'ConvertedComp_Log']].head())

   ConvertedCompYearly  ConvertedComp_Log
0         86155.287263          11.363918
1         86155.287263          11.363918
2         86155.287263          11.363918
3         86155.287263          11.363918
4         86155.287263          11.363918


/tmp/ipykernel_1474/3181982536.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ConvertedComp_Log'] = np.log1p(df['ConvertedCompYearly'])


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [15]:
# SAFETY CHECK
assert 'YearsCodePro' in df.columns, "Column 'YearsCodePro' not found in df"

# Convert YearsCodePro to numeric safely
df['YearsCodePro'] = (
    df['YearsCodePro']
    .replace({
        'Less than 1 year': 0.5,
        'More than 50 years': 51
    })
)

df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# Optional: handle missing values
df['YearsCodePro'] = df['YearsCodePro'].fillna(df['YearsCodePro'].median())

# Define bins and labels
bins = [-1, 2, 5, 10, 100]
labels = ['Junior', 'Intermediate', 'Senior', 'Executive/Expert']

# Create ExperienceLevel column
df['ExperienceLevel'] = pd.cut(
    df['YearsCodePro'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Check distribution
print(df['ExperienceLevel'].value_counts())

ExperienceLevel
Junior              23490
Executive/Expert    18460
Senior              12653
Intermediate        10834
Name: count, dtype: int64


/tmp/ipykernel_1474/560705184.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ExperienceLevel'] = pd.cut(


### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
